In [ ]:
%load_ext autoreload
%autoreload 2

import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
from pathlib import Path
from tqdm import tqdm

import pymupdf4llm

receipts_path = Path("/mnt/c/Users/alexk/Meine Ablage/KFZ/VW ID3/Rechnungen_Laden/")
if not receipts_path.exists():
    receipts_path = Path("/mnt/c/Users/alexk/My Drive/KFZ/VW ID3/Rechnungen_Laden/")
    if not receipts_path.exists():
        raise FileNotFoundError(f"Path {receipts_path} does not exist.")

paths = list(receipts_path.glob("*.PDF")) + list(receipts_path.glob("*.pdf"))

md_texts = []
for path in tqdm(paths):
    md_text = pymupdf4llm.to_markdown(path)
    md_texts.append(md_text)

In [ ]:
import pandas as pd
from datetime import datetime


class BoschReceiptParser:
    @staticmethod
    def parse_md_to_dict(md_text: str):
        lines = md_text.split("\n")
        result = {}
        for line_idx, line in enumerate(lines):
            if line.startswith("#") and "Rechnungsnummer" in line:
                result["invoice_number"] = line.split(" ")[-1].strip()
            if "Rechnungsdatum" in line:
                result["invoice_date"] = pd.Timestamp(datetime.strptime(line.split(" ")[-1].strip(), "%d.%m.%Y"))
            if "Startdatum" in line:
                split = line.split(" ")
                result["start_date"] = split[1]
                result["end_date"] = split[3]
            if "Startzeit" in line:
                split = line.split(" ")
                result["start_time"] = split[1]
                result["end_time"] = split[3]
            if "Gesamtbetrag" in line:
                result["total_amount"] = float(line.split(" ")[-1].strip().replace(",", ".").replace("*", ""))
            if "Charging Station" in line:
                result["charging_station"] = " ".join(lines[line_idx + 2 : line_idx + 4])
            if "BOSCH_CHARGING Ladevorgang - kWh" in line:
                result["kWh"] = line.split(" ")[-3].strip().replace(",", ".")
        result["start"] = pd.Timestamp(
            datetime.strptime(result["start_date"] + " " + result["start_time"], "%d.%m.%Y %H:%M:%S")
        )
        result["end"] = pd.Timestamp(
            datetime.strptime(result["end_date"] + " " + result["end_time"], "%d.%m.%Y %H:%M:%S")
        )
        result["duration"] = result["end"] - result["start"]
        result["Eur/kWh"] = float(result["total_amount"]) / float(result["kWh"])
        for key in ["start_date", "start_time", "end_date", "end_time"]:
            result.pop(key)
        return result


In [ ]:
class MVVReceiptParser:
    @staticmethod
    def parse_md_to_dict(md_text: str):
        result = {}
        lines = [l for l in md_text.split("\n") if l.strip()]
        start_lines = lines[:30].copy()
        for line in start_lines:
            if line.startswith("**Datum:**"):
                result["invoice_date"] = pd.Timestamp(
                    datetime.strptime(line.split(" ")[-1].replace("*", ""), "%d.%m.%Y")
                )
            if line.startswith("**Rechnungsnummer"):
                result["invoice_number"] = line.split(" ")[-1].replace("*", "")
        lines = lines[lines.index("### **Einzelverbindungsnachweis**") :]
        result["charging_station"] = lines[1]
        for line_idx, line in enumerate(lines):
            if line.startswith("**Datum:*"):
                result["invoice_date"] = pd.Timestamp(datetime.strptime(line.split(" ")[-1].strip(), "%d.%m.%Y"))
            if line.startswith("Strombezug"):
                split = line.split(" ")
                result["start_date"] = split[1]
                result["start_time"] = split[2]
                result["end_date"] = split[4]
                result["end_time"] = split[5]
                result["total_amount"] = float(split[12].strip().replace(",", "."))
                result["kWh"] = float(split[6].strip().replace(",", "."))
        result["start"] = pd.Timestamp(
            datetime.strptime(result["start_date"] + " " + result["start_time"], "%d.%m.%Y %H:%M")
        )
        result["end"] = pd.Timestamp(datetime.strptime(result["end_date"] + " " + result["end_time"], "%d.%m.%Y %H:%M"))
        result["duration"] = result["end"] - result["start"]
        result["Eur/kWh"] = float(result["total_amount"]) / float(result["kWh"])
        for key in ["start_date", "start_time", "end_date", "end_time"]:
            result.pop(key)
        return result


In [ ]:
import re
import json


def enbw_to_bosch_format(transactions, invoice_number=None, invoice_date=None, charging_station=None):
    converted = []
    for t in transactions:
        # Parse start and end timestamps
        start = pd.Timestamp(datetime.strptime(t["Start"], "%d.%m.%Y %H:%M:%S"))
        end = pd.Timestamp(datetime.strptime(t["Ende"], "%d.%m.%Y %H:%M:%S"))
        duration = end - start

        # Extract kWh and total amount
        kwh = float(t["Menge"].replace("kWh", "").replace(",", ".").strip())
        total_amount = float(t["Bruttobetrag"].replace("€", "").replace(",", ".").strip())

        # Compose dict in Bosch format
        entry = {
            "invoice_number": invoice_number if invoice_number else "",
            "invoice_date": invoice_date if invoice_date else start.normalize(),
            "charging_station": charging_station if charging_station else t.get("Ladepunkt ID", ""),
            "kWh": f"{kwh:.3f}",
            "total_amount": total_amount,
            "start": start,
            "end": end,
            "duration": duration,
            "Eur/kWh": total_amount / kwh if kwh else None,
        }
        converted.append(entry)
    return converted


def parse_enbw_transactions(md_text: str) -> list:
    """
    Parses an EnBW mobility+ markdown file to extract transaction details.

    Args:
        file_path (str): The path to the markdown file.

    Returns:
        list: A list of dictionaries, where each dictionary represents
              a single transaction with its details. Returns an empty
              list if the file can't be read or the section is not found.
    """
    # --- 1. Isolate the relevant section of the document ---
    try:
        # The section with transactions starts with this header.
        start_marker = "Ihre Einzelladevorgänge im Detail"
        # The section ends just before this note in the footer.
        end_marker = "Die Grundgebühr wird tagesgenau berechnet"

        start_index = md_text.index(start_marker)

        # Find the end of the section; if the marker isn't found, read to the end.
        end_index = md_text.find(end_marker, start_index)
        if end_index == -1:
            transactions_section = md_text[start_index:]
        else:
            transactions_section = md_text[start_index:end_index]

    except ValueError:
        print(f"Error: Section '{start_marker}' not found in the file.")
        return []

    # --- 2. Split the section into individual transaction blocks ---
    # Each transaction detail block is preceded by a table header.
    # We can use this header as a separator to split the text.
    separator = "Nr Datum Adresse Ladeart Kostenart Menge Einheit Nettobetrag Bruttobetrag"
    transaction_blocks = transactions_section.split(separator)

    # The first element after splitting is the text before the first real transaction, so we skip it.
    transaction_blocks = transaction_blocks[1:]

    # --- 3. Parse each block to extract the required data ---
    parsed_transactions = []
    for i, block in enumerate(transaction_blocks, 1):
        transaction_data = {"Transaktion": i}

        # Use regular expressions to find each piece of information.
        # The patterns are designed to be specific but flexible enough for variations.

        # Pattern for: Start: 22.02.2025 15:49:00
        start_match = re.search(r"Start:\s*([\d\.\s:]+)", block)
        if start_match:
            transaction_data["Start"] = start_match.group(1).strip()

        # Pattern for: Ende: 22.02.2025 16:26:48
        end_match = re.search(r"Ende:\s*([\d\.\s:]+)", block)
        if end_match:
            transaction_data["Ende"] = end_match.group(1).strip()

        # Pattern for: Ladepunkt-ID: DE*EBW*E904089*2
        ladepunkt_match = re.search(r"Ladepunkt-ID:\s*(\S+)", block)
        if ladepunkt_match:
            transaction_data["Ladepunkt ID"] = ladepunkt_match.group(1).strip()

        # Pattern for the line containing Menge and Bruttobetrag
        # e.g., "DC Energie 36,55 kWh 15,04 € 17,90 €"
        # Captures the kWh value and the final Euro value on the line.
        charge_match = re.search(r"(?:.*?)(\d+,\d+)\s*kWh\s+[\d,]+\s*€\s+(\d+,\d+\s*€)\s*$", block, re.MULTILINE)
        if charge_match:
            transaction_data["Menge"] = charge_match.group(1).replace(",", ".") + " kWh"
            transaction_data["Bruttobetrag"] = charge_match.group(2).strip()

        # Add the extracted data to our list if it's a valid entry
        if len(transaction_data) > 1:
            parsed_transactions.append(transaction_data)

    return parsed_transactions


class ENBWReceiptParser:
    @staticmethod
    def parse_md_to_dict(md_text: str):
        transactions = parse_enbw_transactions(md_text)
        invoice_number_match = re.search(r"Rechnungs[- ]Nr\.\s*([\d ]+)", md_text)
        invoice_number = invoice_number_match.group(1).replace(" ", "") if invoice_number_match else ""
        invoice_date_match = re.search(r"# Ihre EnBW mobility\+ Rechnung (\d{1,2}\. \w+ \d{4})", md_text)
        if invoice_date_match:
            try:
                invoice_date = pd.to_datetime(invoice_date_match.group(1), dayfirst=True)
            except Exception:
                invoice_date = None
        else:
            invoice_date = None
        converted = enbw_to_bosch_format(transactions, invoice_number=invoice_number, invoice_date=invoice_date)
        return converted


p = "/mnt/c/Users/alexk/My Drive/KFZ/VW ID3/Rechnungen_Laden/Rechnung_2025_02_DE_180113684560.pdf"
for path, md_text in zip(paths, md_texts):
    if str(path) != p:
        continue
    print(path)
    Path("/mnt/c/Users/alexk/My Drive/KFZ/VW ID3/Rechnungen_Laden/test.md").write_text(md_text)
    res = ENBWReceiptParser.parse_md_to_dict(md_text)

In [ ]:
df = []
for path, md_text in zip(paths, md_texts):
    if path.name.startswith("IhreRechnung_"):
        res = BoschReceiptParser().parse_md_to_dict(md_text)
    elif path.name.startswith("mvv_"):
        res = MVVReceiptParser().parse_md_to_dict(md_text)
    elif path.name.startswith("Rechnung_"):
        res = ENBWReceiptParser().parse_md_to_dict(md_text)
    else:
        raise ValueError(f"Unknown receipt format for {path.name}")
    if isinstance(res, list):
        df += res
    else:
        df.append(res)
df = pd.DataFrame(df)
df.sort_values("start", inplace=True)

In [ ]:
start_date = datetime.strptime("2024-10-01", "%Y-%m-%d")
end_date = datetime.strptime("2025-06-01", "%Y-%m-%d")
df_range = df[(df["start"] >= start_date) & (df["start"] <= end_date)]
print(df_range.columns)
total = df_range["total_amount"].sum()
start = df_range.start.min().date()
end = df_range.start.max().date()
days = (end - start).days
summary = f"Over {days} days, you spent {total:.2f} EUR. That's {total / days * 30:.2f} EUR per month."
print(summary)
df_range


In [ ]:
save_path = Path(receipts_path, "accumulated", f"rechnungen_{start}_{end}.csv")
save_path.parent.mkdir(parents=True, exist_ok=True)
print(save_path)
with open(save_path.with_name(f"rechnungen_{start}_{end}_statistik.txt"), "w") as hd:
    hd.write(summary)
df.to_csv(save_path, index=False)